In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv
/kaggle/input/glove6b100dtxt/glove.6B.100d.txt


In [2]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
review=df["review"].values

In [5]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [6]:
import re
from nltk.corpus import stopwords
import string

In [7]:
def clean_review(docs):
    review_list=[]
    for review in docs:
        tokens=review.split()
        re_punc=re.compile('[%s]'%re.escape(string.punctuation))
        tokens=[re_punc.sub('',w ) for w in tokens]
        tokens=[word for word in tokens if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        tokens=[w for w in tokens if not w in stop_words]
        review_list.append(tokens)
    return review_list

In [8]:
review_list=clean_review(review)

In [9]:
embeddings_index=dict()
f=open('/kaggle/input/glove6b100dtxt/glove.6B.100d.txt',mode='rt',encoding='utf-8')
for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:],dtype='float32')
  embeddings_index[word]=coefs
f.close()

In [10]:
words=['man','women','king','queen','boy','girl','phone','laptop','sunday','holiday','festival']

m=np.zeros((11,11))
for i,w1 in enumerate(words):
    u=embeddings_index.get(w1)
#     print(i)
    for j,w2 in enumerate(words):
        m[i][j]=np.dot(u,embeddings_index.get(w2))

In [11]:
np.savetxt("try.csv", m, delimiter=",")

In [12]:
from keras.preprocessing.text import Tokenizer

In [13]:
t=Tokenizer()
t.fit_on_texts(review_list)

In [14]:
vocab_size=len(t.word_index)+1

In [15]:
encoded_review=t.texts_to_sequences(review_list)

In [16]:
a=0
for i in range(len(encoded_review)):
    a=a+len(encoded_review[i])

print("average length of rewiew",a/len(encoded_review))

average length of rewiew 132.00926


In [17]:
from keras.preprocessing.sequence import pad_sequences

In [18]:
max_length=150
padded_review=pad_sequences(encoded_review,maxlen=max_length,padding='post')

In [19]:
print(len(encoded_review[0]),padded_review[0])

184 [ 2441  5933  5515  1394   303   507    38  3324   272   260 23136     2
    11   392  3245 11304   258 15744  6724  2475   987 60444    11  2569
  1296 24969   460  4625  2452  1129  7000  2922 12783   329 60445 17229
   237  4992  3632   460   264 79910  8297 40446 15110  5063  7730  2397
 18096 60446   247  9077  7317 13206  8545 34385    42   143  5319     2
     1    12    57   188  1224    51   590   105   180   174   472  2931
   740    97  1192  4255  2432  1022   740  1335   740 79911    71   904
   100     3    24   315     1    53   117  3176  1501  2150     1   318
    57     1  1477     1   196     1  1387  1172  3245   102 10076   237
  2023  2031   507   162   507  7801  7001  4869 13930  2900 32110  6884
 13930   416   545    19   159    18  9740   675   737  6884   582  1129
 20255   590   473   857  1927  1129   482    69  3245   113   332  3690
  3196 79912    19  1132  3925   425]


In [20]:
from sklearn.model_selection import train_test_split

In [21]:
sentiment=(df['sentiment'].values=="positive")*1

In [22]:
sentiment

array([1, 1, 1, ..., 0, 0, 0])

In [23]:
Xtrain, Xtest, ytrain,ytest=train_test_split(padded_review,sentiment)

In [24]:
print(Xtrain.shape,Xtest.shape)

(37500, 150) (12500, 150)


In [25]:
embedding_matrix=np.zeros((vocab_size,100))
for word ,i in t.word_index.items():
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector

In [26]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding

In [27]:
model=Sequential()
model.add(Embedding(vocab_size,100,weights=[embedding_matrix],input_length=max_length,trainable=False))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 100)          17507600  
_________________________________________________________________
flatten (Flatten)            (None, 15000)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1920128   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 19,436,049
Trainable params: 1,928,449
Non-trainable params: 17,507,600
_________________________________________________________________


In [29]:
import tensorflow as tf
es=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [30]:
model.fit(Xtrain,ytrain,epochs=10,validation_split=.2,callbacks=[es])

Epoch 1/10
592/938 [=================>............] - ETA: 2s - loss: 0.0205 - acc: 0.9932

KeyboardInterrupt: 

In [ ]:
model.evaluate(Xtest,ytest)

In [ ]:
def pred(doc):
    doc=t.texts_to_sequences(doc)
    doc=pad_sequences(doc,maxlen=max_length,padding='post')
    return model.predict(doc)

In [ ]:
pred(["Death & The City 90 minutes of my life wasted watching SJP walk around NYC in her Louboutins and Manolos.I was hoping her character would die so I could of watched a real movie."])

In [ ]:
pred(["Omg I want the time I spent watching this movie, if you can call it that, back. It was such a waste and it ended with no conclusion... NONE! She wandered around the city aimlessly(felt like it) and had on the same outfit for 98% of the movie. I very seldom review movies, but after watching this one I had to. Smh, just pass on it and watch something not this. #theworst"])